In [2]:
import torch
from torch.utils.data import DataLoader

from datautil.waymo_dataset import WaymoDataset, waymo_collate_fn

dataset = WaymoDataset('data')
dataloader = DataLoader(dataset, batch_size=4, collate_fn=lambda x: waymo_collate_fn(x), shuffle=False)

data0 = next(iter(dataloader))

In [17]:
import torch
import torch.nn as nn
from model.utils import *

state_feat, agent_batch_mask, road_feat, traffic_light_feat = data0

in_feat_dim = 9
feature_dim = 256
in_dynamic_rg_dim = 3
time_steps = 91
head_num = 4

layer_A = nn.Sequential(nn.Linear(in_feat_dim,feature_dim), Permute4Batchnorm((0,2,1)),
                            nn.BatchNorm1d(feature_dim), Permute4Batchnorm((0,2,1)))

layer_B = nn.Sequential(nn.Linear(in_dynamic_rg_dim,feature_dim), Permute4Batchnorm((0,2,1)),
                            nn.BatchNorm1d(feature_dim), Permute4Batchnorm((0,2,1)))


A_ = layer_A(state_feat)
B_ = layer_B(traffic_light_feat)

In [18]:
viewmodule_ = View((-1,time_steps,head_num, int(feature_dim/head_num)))
layer_K_ = nn.Sequential(nn.Linear(feature_dim,feature_dim), nn.ReLU(), viewmodule_)

In [22]:
print(A_.shape)
layer_K_(A_).shape

torch.Size([175, 91, 256])


torch.Size([175, 91, 4, 64])